In [1]:
include("functions.jl")

genetique

In [ ]:
# Définition des instances de base
@static if !@isdefined(instance_list)
    const instance_list = (
        read_instance("graphs/flat300_26_0.col", 26),
        read_instance("graphs/le450_15c.col", 15),
        read_instance("graphs/dsjc125.1.col", 5),
        read_instance("graphs/dsjc125.9.col", 44),
        read_instance("graphs/dsjc250.1.col", 8),
        read_instance("graphs/dsjc250.9.col", 72),
        read_instance("graphs/dsjc250.5.col", 28),
        read_instance("graphs/dsjc1000.5.col", 86),
        read_instance("graphs/dsjc1000.5.col", 85),
        read_instance("graphs/dsjc1000.5.col", 84))
end

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [2]:
# Comparaison d'une solution aléatoire et d'une solution gloutonne
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println(instance.name,"\tk=",instance.k)
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=5
SOLUTION ALÉATOIRE:


[4, 4, 4, 4, 3, 1, 4, 2, 2, 3, 3, 2, 4, 2, 5, 4, 2, 3, 4, 2, 5, 3, 2, 2, 4, 5, 4, 5, 3, 5, 1, 4, 3, 1, 3, 1, 5, 1, 4, 5, 5, 3, 4, 1, 3, 5, 5, 5, 4, 1, 5, 1, 2, 4, 2, 4, 3, 4, 5, 1, 1, 2, 2, 4, 1, 5, 3, 4, 2, 3, 4, 5, 1, 2, 2, 2, 1, 2, 3, 1, 2, 2, 5, 2, 3, 1, 5, 1, 1, 1, 2, 2, 4, 3, 4, 5, 1, 1, 5, 5, 1, 5, 3, 2, 1, 5, 2, 2, 1, 2, 2, 5, 1, 2, 3, 5, 1, 4, 5, 4, 5, 4, 4, 5, 1]
collisions: 129



SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [3]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println(
            instance.name," k=",instance.k,"\t",
            minimum(conflicts_samples),"\t",
            sum(conflicts_samples)/nsamples,"\t",
            maximum(conflicts_samples),"\t",
            sum(time_samples),"\t",
            time_samples[argmin(conflicts_samples)],"\t",
            nsamples/sum(time_samples)
        ) # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND


flat300_26_0.col k=26	226	226.0	226	0.0019939	0.00020030000000000002	5015.296654797132
le450_15c.col k=15	362	362.0	362	0.0022299000000000004	0.00028260000000000004	4484.506031660612
dsjc125.1.col k=5	47	47.0	47	9.28e-5	1.3300000000000001e-5	107758.62068965516
dsjc125.9.col k=44	21	21.0	21	0.0001509	2.01e-5	66269.05235255136
dsjc250.1.col k=8	88	88.0	88	0.00045480000000000005	5.2000000000000004e-5	21987.686895338607
dsjc250.9.col k=72	61	61.0	61	0.0006055000000000001	6.71e-5	16515.276630883563
dsjc250.5.col k=28	85	85.0	85	0.0013737	0.0001444	7279.609812914027


dsjc1000.5.col k=86	378	378.0	378	0.0220395	0.0021957	453.7308015154609


dsjc1000.5.col k=85	398	398.0	398	0.022905900000000003	0.0021576	436.56874429732073


dsjc1000.5.col k=84	412	412.0	412	0.022576900000000004	0.0022228	442.9306060619482

# 2) Structure de voisinage

On choisit d'utiliser un opérateur de voisinage simple:
 - Appliquer la couleur $i$ au nœud $j$ choisis de manière à réduire le plus possible le nombre de conflits par rapport à la solution courante.
   - On choisit le premier couple $(i,j)$ occasionnant la plus forte réduction de conflits.
   - Pour cela, comme suggéré par Daniel Porumbel, on maintient une matrice des conflits stockant pour chaque nœud $j$ et chaque couleur $i$ le nombre de conflits en ce nœud $j$ si l'on lui applique la couleur $i$.

On obtient donc une structure de voisinage qui contient toutes les solutions dont un unique nœud diffère par sa couleur de la solution d'origine.

In [4]:
# Test de descente locale simple
begin
    instance = instance_list[3]
    solution = glouton(instance)

    for t = 1:25
        simple_neighbor(instance,solution)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 43
t: 2	conflits: 40
t: 3	conflits: 38
t: 4	conflits: 36
t: 5	conflits: 34
t: 6	conflits: 32
t: 7	conflits: 30
t: 8	conflits: 28
t: 9	conflits: 27
t: 10	conflits: 26
t: 11	conflits: 25
t: 12	conflits: 24
t: 13	conflits: 23
t: 14	conflits: 22
t: 15	conflits: 21
t: 16	conflits: 20
t: 17	conflits: 19
t: 18	conflits: 18
t: 19	conflits: 17
t: 20	conflits: 16
t: 21	conflits: 15
t: 22	conflits: 15
t: 23	conflits: 15
t: 24	conflits: 15
t: 25	conflits: 15


# 3) Métaheuristique proposée

On choisit d'écrire un algorithme mémétique:

## 0 - Règle de sélection

On utilise la règle de la roue de la fortune, en utilisant comme poids l'objectif dans l'opérateur d'élimination et l'inverse de l'objectif dans la fonction de reproduction.

## 1 - Fonction de reproduction

On génère λ enfants, chacun à partir de deux parents sélectionnés avec la règle ci-dessus, en leur appliquant l'opérateur de croisement.\
Afin de garantir la diversité des solutions, si l'enfant est trop proche d'une solution existante (plus proche que le seuil de rejet, voir opérateur d'élimination), il est rejeté.

### Opérateur de croisement

Comme suggéré par Daniel Porumbel, on choisit un opérateur de croisement qui copie des classes de couleurs entières des parents dans l'enfant (sur les nœuds où l'enfant n'a pas encore de couleur définie), dans l'ordre décroissant de taille de la classe, indifféremment de leur origine parentale.

## 2 - Opérateur d'intensification

On effectue une recherche locale simple, en utilisant la structure de voisinage simple `simple_neighbor` citée ci-dessus.\
On interrompt la recherche locale dès que l'on atteint un minimum local ou un plateau (i.e. si la valeur de l'objectif n'a pas changé). Dans tous les cas, la recherche locale se termine si l'on atteint le nombre maximal d'itérations autorisé.

## 3 - Fonction d'élimination

Comme proposé par Daniel Porumbel, on calcule la distance entre chaque individu de la population.
 - Si la distance minimale est inférieure au seuil de rejet, on élimine la solution de moins bon objectif.
 - Sinon, on utilise la règle de sélection pour obtenir une solution, on calcule son voisin le plus proche, et on élimine celle de moins bon objectif.

### Calcul de distance entre deux solutions

Comme Daniel Porumbel l'a fait remarquer, la distance entre deux solutions n'est pas simplement le nombre de couleurs différentes, car la solution est définie à une permutation des couleurs près. Pour calculer la distance, il est donc nécessaire de déterminer la permutation minimisant le nombre de couleurs différentes entre deux solutions, donc maximisant le nombre de couleurs identiques:

$\displaystyle{\max_{\text{perm}} \sum_{c} S(c,\text{perm}(c))}$

Où $S$ est la matrice de similarité, i.e. $S(a,b)$ est le nombre de nœuds $i$ pour lesquels $x_1(i)=a$ et $x_2(i)=b$.

Le calcul de la meilleure permutation est un problème NP-complet, ce qui exclut l'utilisation d'un algorithme exact pour la déterminer. On écrit donc une heuristique:

Les cases de $S$ contenant les plus grandes valeurs ont de grandes chances d'être atteintes par la permutation. On définit donc la permutation comme suit:
 - $(a,b) = \argmax(S)$
 - $\text{perm}(b) = a$
 - vider la colonne $b$ (correspondant à $\text{perm}(b) = u$, car perm est une fonction) ainsi que la ligne $a$ (correspondant à $\text{perm}(v) = a$, car perm est injective)
 - recommencer

Cette heuristique a toutes les chances de fournir la permutation optimale si les solutions sont effectivement proches, ce qui donne une matrice $S$ avec une case de valeur nettement plus grande que les autres de sa ligne/colonne, donc détectable par l'heuristique. Cela tombe bien, car il est important d'avoir une bonne estimation de la distance pour des solutions proches, alors que pour des solutions éloignées, cela est moins critique.

Test: Tabou

In [ ]:
# Test de descente locale avec tabou
begin
    instance = instance_list[3]
    solution = glouton(instance)

    for t = 1:100
        tabu_neighbor(instance,solution,10,t)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

Algo mémétique

In [6]:
begin
    instance = instance_list[3]

    genetique(instance,25,25,10,100,80)
end

┌ Info: it: 0	temps:0.0s	conflits: [20, 25, 20, 28, 24, 23, 27, 26, 25, 16, 27, 27, 20, 27, 20, 16, 22, 22, 26, 28, 21, 19, 29, 30, 16]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:340
┌ Info: it: 1	temps:0.04s	conflits: [20, 23, 27, 26, 25, 16, 27, 16, 19, 12, 17, 20, 16, 24, 19, 18, 17, 16, 14, 14, 24, 15, 23, 19, 13]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 2	temps:0.08s	conflits: [16, 27, 16, 12, 17, 16, 14, 14, 15, 13, 16, 14, 19, 15, 17, 12, 14, 19, 19, 16, 13, 14, 13, 23, 15]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 3	temps:0.1s	conflits: [16, 16, 12, 13, 14, 12, 14, 13, 14, 13, 17, 12, 12, 11, 14, 15, 13, 15, 10, 11, 14, 16, 17, 15, 12]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 4	temps:0.11s	conflits: [12, 13, 12, 14, 17, 12, 12, 11, 14, 10, 11, 16, 12, 14, 12, 11, 10, 10, 8, 11, 16, 14, 12, 14, 12]
└ @ Main c:\Users\SENEGAS\Document

┌ Info: it: 5	temps:0.15s	conflits: [12, 13, 12, 17, 11, 10, 11, 16, 12, 11, 10, 10, 8, 11, 12, 9, 13, 14, 12, 15, 10, 9, 13, 10, 9]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 6	temps:0.16s	conflits: [12, 12, 11, 10, 8, 11, 12, 9, 13, 14, 10, 10, 9, 10, 13, 9, 8, 12, 11, 7, 10, 8, 7, 12, 8]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 7	temps:0.18s	conflits: [10, 12, 13, 14, 10, 10, 9, 9, 8, 7, 10, 13, 7, 13, 10, 7, 7, 6, 8, 7, 6, 10, 8, 8, 15]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 8	temps:0.2s	conflits: [12, 14, 9, 9, 8, 10, 7, 10, 7, 6, 7, 6, 10, 8, 15, 14, 7, 6, 11, 8, 12, 13, 15, 9, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 9	temps:0.22s	conflits: [10, 7, 6, 7, 6, 10, 8, 7, 6, 11, 13, 9, 8, 11, 16, 7, 8, 6, 8, 9, 10, 13, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 10	tem

┌ Info: it: 11	temps:0.24s	conflits: [6, 6, 6, 6, 6, 7, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6, 6, 7, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 12	temps:0.26s	conflits: [6, 6, 6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6, 7, 6, 6, 6, 7, 7, 6, 6, 6, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 13	temps:0.27s	conflits: [6, 7, 6, 6, 6, 6, 6, 7, 6, 7, 6, 6, 7, 6, 7, 6, 6, 6, 6, 7, 7, 7, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 14	temps:0.28s	conflits: [6, 6, 6, 7, 6, 7, 6, 6, 6, 7, 6, 6, 6, 6, 7, 6, 6, 6, 6, 7, 6, 6, 6, 7, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 15	temps:0.3s	conflits: [6, 6, 7, 6, 7, 6, 6, 6, 6, 7, 6, 6, 6, 6, 6, 7, 6, 6, 7, 7, 6, 6, 7, 6, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 16	temps:0.31s	conflits: [6, 6, 7, 6, 6, 7, 6, 6, 7, 6, 6, 6, 7, 6, 7

┌ Info: it: 19	temps:0.35s	conflits: [7, 6, 7, 7, 6, 6, 7, 7, 7, 6, 6, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 20	temps:0.37s	conflits: [7, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 21	temps:0.38s	conflits: [6, 6, 6, 6, 6, 7, 6, 7, 7, 7, 7, 7, 6, 7, 6, 7, 7, 7, 7, 7, 7, 7, 6, 7, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 22	temps:0.4s	conflits: [6, 6, 6, 6, 6, 7, 7, 6, 7, 7, 7, 6, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 6, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 23	temps:0.41s	conflits: [6, 6, 6, 6, 6, 6, 7, 7, 6, 7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 6, 7, 7, 7, 7, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 24	temps:0.42s	conflits: [6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7

┌ Info: it: 27	temps:0.46s	conflits: [6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 6, 7, 7, 6, 7, 6, 7, 7, 7, 6, 7, 7, 6, 7, 7]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 28	temps:0.48s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 6, 6, 6, 7, 7, 6, 7, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 29	temps:0.49s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 30	temps:0.5s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 31	temps:0.51s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 32	temps:0.53s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6

...

┌ Info: it: 75	temps:1.1s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 76	temps:1.11s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 77	temps:1.13s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 78	temps:1.14s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 79	temps:1.16s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 80	temps:1.17s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6

┌ Info: it: 84	temps:1.22s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 85	temps:1.23s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 86	temps:1.25s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 87	temps:1.26s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 88	temps:1.27s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 89	temps:1.28s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 

┌ Info: it: 91	temps:1.31s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 92	temps:1.32s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 93	temps:1.33s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 94	temps:1.34s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 95	temps:1.36s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347
┌ Info: it: 96	temps:1.37s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 

┌ Info: it: 100	temps:1.43s	conflits: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
└ @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\functions.jl:347


Solution([4, 1, 4, 1, 1, 3, 3, 2, 4, 5  …  1, 2, 2, 3, 1, 3, 3, 2, 1, 1], [3 0 … 0 0; 3 3 … 5 2; … ; 0 1 … 2 2; 1 2 … 2 1], [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 6)